# Naive forecasting

Forecasting without incorporation of external forecasts from CBS, NBC, ESPN, etc.

In [2]:
import pandas as pd

data = pd.read_csv("../../data/raw/pro_football_reference.csv", index_col=0)

data.head()

,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,...,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,draftkings_points,fanduel_points,vbd,fantasy_rank_pos,fantasy_rank_overall,year
0,Barry Sanders,DET,RB,29.0,16.0,16.0,0.0,0.0,0.0,0.0,...,NaN,NaN,320.0,352.8,358.8,336.3,186.0,1,1.0,1997
1,Terrell Davis,DEN,RB,25.0,15.0,15.0,0.0,0.0,0.0,0.0,...,3.0,NaN,300.0,341.7,347.7,320.7,166.0,2,2.0,1997
2,Dorsey Levens,GNB,RB,27.0,16.0,16.0,0.0,0.0,0.0,0.0,...,1.0,NaN,255.0,307.5,313.5,281.0,121.0,3,3.0,1997
3,Jerome Bettis,PIT,RB,25.0,15.0,15.0,0.0,0.0,0.0,0.0,...,NaN,NaN,232.0,246.5,252.5,239.0,98.0,4,4.0,1997
4,Brett Favre,GNB,QB,28.0,16.0,16.0,304.0,513.0,3867.0,35.0,...,NaN,1.0,291.0,289.4,311.4,305.4,96.0,1,5.0,1997
